In [12]:
import TradeClient
import asyncio
import nest_asyncio
import Analysis_new
import MarketDataFetcher
import DataStoreage

nest_asyncio.apply()

ins_market = MarketDataFetcher.FuturesMarket()

symbols = ['BTCUSDT', 'TRXUSDT', 'XRPUSDT']
data = {}
for symbol in symbols:
    data[symbol] = DataStoreage.KlineData()
    kline_data = asyncio.run(ins_market.fetch_klines_limit(symbol=symbol, interval='3m', limit=10))
    data[symbol].initialize_data(kline_data)

ins_analysis = Analysis_new.AnalysisManager(data)

AttributeError: module 'Analysis_new' has no attribute 'AnalysisManager'

In [9]:
data['BTCUSDT'].interval_3m

[[1737986580000,
  '100479.50',
  '100540.00',
  '100359.80',
  '100359.80',
  '609.893',
  1737986759999,
  '61287271.96150',
  12582,
  '294.528',
  '29598171.99740',
  '0'],
 [1737986760000,
  '100359.90',
  '100480.00',
  '100232.70',
  '100467.00',
  '594.660',
  1737986939999,
  '59665987.60170',
  11824,
  '366.785',
  '36804476.63560',
  '0'],
 [1737986940000,
  '100467.10',
  '100468.90',
  '100200.00',
  '100332.00',
  '455.166',
  1737987119999,
  '45667043.88950',
  11542,
  '193.462',
  '19407976.69270',
  '0'],
 [1737987120000,
  '100332.10',
  '100343.10',
  '100050.00',
  '100102.40',
  '516.344',
  1737987299999,
  '51731311.72850',
  14807,
  '211.994',
  '21242909.97960',
  '0'],
 [1737987300000,
  '100102.40',
  '100400.00',
  '100102.40',
  '100330.80',
  '648.882',
  1737987479999,
  '65061126.17800',
  13615,
  '374.915',
  '37595360.47380',
  '0'],
 [1737987480000,
  '100330.70',
  '100516.70',
  '100200.00',
  '100498.50',
  '426.774',
  1737987659999,
  '42847